# Getting coordinates from 2gis,yandex,google

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\93.0.4577.15\\chromedriver.exe')
#urls=['https://yandex.kz/maps/','https://www.google.com/maps/','https://2gis.kz/']
urls=['https://yandex.kz/maps/']
google={}
yandex={}
gis={}
for url in urls:
    driver.get(url)
     
    if url=='https://www.google.com/maps/':
        for i in range(len(address_list)):
            searchBox=driver.find_element_by_xpath('//*[@id="searchboxinput"]')    
            searchBox.send_keys(str(address_list[i]))
            searchBox.send_keys(Keys.ENTER)
            time.sleep(3)
            g=driver.current_url
            g_lon=g.split('/@')[1].split('z/')[0].split(',')[1]
            g_lat=g.split('/@')[1].split('z/')[0].split(',')[0]    
            google[str(address_list[i])]=str(g_lon)+' , '+str(g_lat)
            time.sleep(1)
            searchBox.clear()
    elif url=='https://yandex.kz/maps/':                    
        for i in range(len(address_list)):
            searchBox=driver.find_element_by_class_name('input__control')    
            searchBox.send_keys(str(address_list[i]))   
            searchBox.send_keys(Keys.ENTER) 
            time.sleep(3) 
            y=driver.current_url
            #time.sleep(2) 
            if len(y)<100:
                y_lon=y.split('?ll=')[1].split('%2C')[0]
                y_lat=y.split('?ll=')[1].split('%2C')[1].split('&')[0]
                yandex[str(address_list[i])]=str(y_lon)+' , '+str(y_lat)

            else:
                y_lon=y.split('ll=')[1].split('%2C')[0]
                y_lat=y.split('?ll=')[1].split('%2C')[1].split('&')[0]
                
                yandex[str(address_list[i])]=str(y_lon)+' , '+str(y_lat)
            for j in range(len(address_list[i])+5):
                searchBox.send_keys(Keys.BACK_SPACE)
                #driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div/div/div/div/form/div[5]/button').click()

            
            
    else:
        for i in range(len(address_list)):
            try:
                searchBox=driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[2]/form/div/input')    
                searchBox.send_keys(str(address_list[i]))
                searchBox.send_keys(Keys.ENTER)
                #time.sleep(3)
                #driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div[3]/div/div[1]').click()
                time.sleep(5)
                gis2=driver.current_url
                gis_lon=gis2.split('m=')[1].split('%2C')[0]
                gis_lat=gis2.split('m=')[1].split('%2C')[1].split('%2F')[0] 
                gis[str(address_list[i])]=str(gis_lon)+' , '+str(gis_lat)                
                driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[2]/form/div/div').click()
                #time.sleep(3) 
            except NoSuchElementException:
                continue            
        
driver.close()          
          
                

In [ ]:
dataframe1=pd.DataFrame(yandex.items(),columns=['Address','Longitude and Latitude'])
#dataframe2=pd.DataFrame(google.items(),columns=['Address','Longitude and Latitude'])
#dataframe3=pd.DataFrame(gis.items(),columns=['Address','Longitude and Latitude'])
dataframe1[['lon(yandex)','lat(yandex)']] = dataframe1['Longitude and Latitude'].str.split(",",expand=True)
#dataframe2[['lon(google)','lat(google)']] = dataframe2['Longitude and Latitude'].str.split(",",expand=True)
#dataframe3[['lon(2gis)','lat(2gis)']] = dataframe3['Longitude and Latitude'].str.split(",",expand=True)
dataframe1.drop('Longitude and Latitude',axis=1,inplace=True)
#dataframe2.drop('Longitude and Latitude',axis=1,inplace=True)
#dataframe3.drop('Longitude and Latitude',axis=1,inplace=True)

# Getting boundring coordinates of any country/region from json(yandex.map)

In [ ]:
import os
import json
import pandas as pd
base_dir = 'C:\\Users\\svnduw\\Desktop\\Bekbol\\new region\\03.09.2021_'
address_data={}

for file in os.listdir(base_dir):
    with open(base_dir+'//'+file,encoding="utf8") as f:
        data=json.load(f)
        coordinates=[]
        for i in range(len(data['data']['items'][0]['displayGeometry']['geometries'])):
            for j in range(len(data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'])):
                for k in range(len(data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'][j])):            
                    for z in range(len(data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'][j][k])):
                        coordinates_by_parts=data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'][j][k][z]
                        coordinates.append(coordinates_by_parts)
        address_data[file.split('.')[0]]=str(coordinates)  

In [ ]:
import json
import pandas as pd
from shapely.geometry import Polygon
f=open("C:\\Users\\svnduw\\Desktop\\Bekbol\\Merke.json", encoding='UTF8')
data = json.load(f)
coordinates=[]
for i in range(len(data['data']['items'][0]['displayGeometry']['geometries'])):
    for j in range(len(data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'])):
        for k in range(len(data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'][j])):            
            for z in range(len(data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'][j][k])):
                           coordinates_by_parts=data['data']['items'][0]['displayGeometry']['geometries'][i]['coordinates'][j][k][z]
                           coordinates.append(coordinates_by_parts)
x=[]
y=[]
for i in range(len(coordinates)):
    if(i%2==0):
        x.append(coordinates[i])
    else:
        y.append(coordinates[i])
df = pd.DataFrame(list(zip(x, y)),columns=['lat','lon'])